# 事务

`fsspec` 支持事务(Transactions)，在事务期间，对远程存储中的文件的写入被推迟（通常放入临时位置），直到事务结束，此时整个事务以半原子方式完成，所有文件都被移动/提交到它们的最终目的地。具体实现是文件系统特定的（并非所有都支持），但核心思想是，要么所有文件都写入，要么都不写入，以减轻数据损坏的风险。这个功能可以像下面这样使用：

```python
fs = fsspec.filesystem(...)
with fs.transaction:
    with fs.open('file1', 'wb') as f:
        f.write(b'some data')
    with fs.open('file2', 'wb') as f:
        f.write(b'more data')
```

在这里，`file1` 和 `file2` 在事务上下文完成之前不会被移动到目标位置。如果上下文由于（未捕获的）异常而结束，那么这些文件将被丢弃，并且文件的目标位置保持未被触碰。

{class}`fsspec.spec.Transaction` 类允许对操作进行微调，每个 `fsspec` 实例都有一个此类的实例作为属性 `.transaction` 来提供访问。

请注意，同步多个实例之间的事务，可能是跨集群的，是一个更难解决的问题，这里描述的实现只是解决方案的一部分。